In [1]:
import os, glob
import cv2
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [3]:
img_width, img_height = 150, 150
batch_size = 32

# Create an ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    # shear_range=0.2,
    # zoom_range=0.2,
    horizontal_flip=True
)

train_path = r"D:\datasets\deepfakes_imgs\Dataset\Sampled_train"

In [4]:
train_generator = train_datagen.flow_from_directory(
    train_path,  # Path to the parent directory containing class folders
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'  # Assuming you have two classes
)


Found 70002 images belonging to 2 classes.


In [6]:

# Define your CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    
    Dense(128, activation='relu'),
    Dense(10, activation = 'relu'),
    Dense(2, activation='softmax')
    
])



In [8]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 17, 17, 128)     

In [11]:
# Train the model using fit_generator
with tf.device('/GPU:0'):
    history = model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=5
    )

Epoch 1/5


C:\Users\Siddhartha Devan V\AppData\Local\Temp\ipykernel_10336\3145427923.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


2187/2187 [==============================] - 163s 74ms/step - loss: 0.1994 - accuracy: 0.9157
Epoch 2/5
2187/2187 [==============================] - 117s 54ms/step - loss: 0.1523 - accuracy: 0.9362
Epoch 3/5
2187/2187 [==============================] - 185s 84ms/step - loss: 0.1279 - accuracy: 0.9471
Epoch 4/5
2187/2187 [==============================] - 173s 79ms/step - loss: 0.1104 - accuracy: 0.9547
Epoch 5/5
2187/2187 [==============================] - 154s 71ms/step - loss: 0.0973 - accuracy: 0.9607


In [12]:
print(train_generator.class_indices)


{'fake_sampled': 0, 'real_sampled': 1}


In [15]:
test_datagen = ImageDataGenerator(
    rescale=1. / 255
)

test_path = r"D:\datasets\deepfakes_imgs\Dataset\Test"

In [46]:
test_generator = test_datagen.flow_from_directory(
    test_path,  # Path to the parent directory containing class folders
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',  # Assuming you have two classes
    shuffle = False
)


Found 10905 images belonging to 2 classes.


In [47]:
test_generator.class_indices

{'Fake': 0, 'Real': 1}

In [19]:
model.save("model_1")

INFO:tensorflow:Assets written to: model_1\assets


INFO:tensorflow:Assets written to: model_1\assets


In [48]:
res = model.evaluate_generator(test_generator)

C:\Users\Siddhartha Devan V\AppData\Local\Temp\ipykernel_10336\9663080.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  res = model.evaluate_generator(test_generator)


In [49]:
print(res)

[0.2908989489078522, 0.8818890452384949]


In [50]:
Y_test  = test_generator.classes

In [51]:
print(len(Y_test))

10905


In [52]:
print(Y_test[6000:6040])

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1]


In [53]:
Y_pred = model.predict_generator(test_generator)

C:\Users\Siddhartha Devan V\AppData\Local\Temp\ipykernel_10336\3358749156.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model.predict_generator(test_generator)


In [54]:
Y_pred_labels = [np.argmax(i) for i in Y_pred]

In [55]:
print(Y_pred_labels)

[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [56]:
from sklearn.metrics import classification_report

In [57]:
report = classification_report(Y_test, Y_pred_labels)
print(report)

              precision    recall  f1-score   support

           0       0.90      0.87      0.88      5492
           1       0.87      0.90      0.88      5413

    accuracy                           0.88     10905
   macro avg       0.88      0.88      0.88     10905
weighted avg       0.88      0.88      0.88     10905



In [13]:
import tensorflow as tf

# Check if GPU is available
if tf.test.is_gpu_available():
    print("GPU is available")
    # Additional information about the GPU
    print(tf.config.experimental.list_physical_devices('GPU'))
else:
    print("GPU is NOT available")

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
